# A computational companion to Alan Turing's paper *On the Chemical Basis of Morphogenesis* (1952)


In [2]:
import numpy as np
from sympy import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# 3. Chemical Reaction
Suppose the reaction
$ A \rightarrow B $
is catalysed by G so we have
$ A + G \rightleftharpoons C \rightarrow B + G $

By the law of mass action,

$\frac{d[A]}{dt} = -k_1[A][G] + k_2[C] \\
\frac{d[B]}{dt} = k_3[C] \\
\frac{d[C]}{dt} = k_1[A][G]-k_2[C] -k_3[C] \\
\frac{d[G]}{dt} = -k_1[A][G] + k_2[C] + k_3[C] $

reaction graph diagram here
Turing points out we can simplify these kinetics if $k_3$ is small, so $ C \rightarrow B + G $ is much slower than $A + G \rightleftharpoons C$. In that case we have an approximate equilibrium where 
$\frac{d[A]}{dt} \approx \frac{d[G]}{dt} \approx 0 \approx -k_1[A][G] + k_2[C]$

so
$[A][G] \approx \frac{k_2}{k_1}[C]$


In [ ]:
z

## 4. The breakdown of symmetry and homogeneity
dx = 5*X + 6*Y + 1

In [3]:
r = 1
a=1; b=1; c=1; d=1
mu = 1 ; nu = 1

$(p-a + \mu s^2 / r^2)(p-d+ \nu s^2 / r^2) = bc$

In [1]:
p,A,B = symbols('p A B')
C=D=1
for theta in range(0,2*pi):
    for s in range(-100,100):
        p0,p1=solveset((p-a+mu*s**2/r**2)*(p-d+nu*s**2/r**2) - b * c,p)
        (As,Bs),=linsolve([A*(p0-a+mu*s**2/r**2) - b*C,
                           B*(p1-a+mu*s**2/r**2) - b*D], A,B)
        x = As * np.sum(exp(p0*t) + Bs*exp(p1*t)*exp(i*s*theta))

NameError: name 'symbols' is not defined